# Otimização de Estrutura Cristalina usando MACE e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
from ase.io import write, read

import numpy as np
%matplotlib inline

## Fazendo simulação usando MACE_MP

In [2]:
from mace.calculators import mace_mp

macemp = mace_mp(model="small", dispersion=True, default_dtype = "float32", device="cuda")

/home/elvis/Programs/miniforge3/envs/mlff/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


Using Materials Project MACE for MACECalculator with /home/elvis/.cache/mace/20231210mace128L0_energy_epoch249model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/home/elvis/Programs/miniforge3/envs/mlff/lib/python3.10/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.
Using TorchDFTD3Calculator for D3 dispersion corrections


### Criando a geometria da rede cristalina

Pode ser um arquivo cif, por exemplo

> atoms = read('blabla.cif')

In [3]:
from ase.lattice.cubic import FaceCenteredCubic

# bulk system
atoms = FaceCenteredCubic(size=(1, 1, 1), symbol='Pt', latticeconstant=4.0, pbc=True)

### Definindo as simetrias a serem mantidas

In [ ]:
from ase.constraints import FixSymmetry
from ase.filters import UnitCellFilter

atoms.set_constraint(FixSymmetry(atoms)) #Constraint to preserve spacegroup symmetry during optimisation.
atoms_filter = UnitCellFilter(atoms) #Modify the supercell and the atom positions.

### Definindo o método de otimização e a calculadora

In [ ]:
atoms.calc = macemp

from ase.optimize import BFGS

opt = BFGS(atoms_filter)

### Calculando a otimização com BFGS

In [6]:
print('Initial Energy', atoms.get_potential_energy())
opt.run(fmax=0.001)
print('Final Energy', atoms.get_potential_energy())

Initial Energy -27.160777170256566
      Step     Time          Energy          fmax
BFGS:    0 10:54:02      -27.160777        1.064954


/home/elvis/Programs/miniforge3/envs/mlff/lib/python3.10/site-packages/torch_dftd/torch_dftd3_calculator.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  cell: Optional[Tensor] = torch.tensor(


BFGS:    1 10:54:02      -27.203802        0.818386
BFGS:    2 10:54:03      -27.261398        0.087236
BFGS:    3 10:54:03      -27.261987        0.006242
BFGS:    4 10:54:03      -27.261993        0.000051
Final Energy -27.261992681471707


### Pegando novos parâmetros da rede

In [7]:
atoms.cell.cellpar()

array([ 3.93882994,  3.93882994,  3.93882994, 90.        , 90.        ,
       90.        ])